In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
from scipy.spatial.distance import mahalanobis as mah_dist # u, v, cov_mat
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))


def po_linear_model(graph, alpha=1, beta=1, sigma=0.1, gamma=2):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i]  

def po_multiplicative_model(graph, alpha=1, sigma=0.1, delta=1, gamma=2): 
    for i in graph.nodes:
        graph.nodes[i]["y"] = ( (alpha + sigma * np.random.normal()) * graph.degree[i]/avg_deg )  *  (1 + delta * graph.nodes[i]["z"] + gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]]) / len(graph[i]) )

DGL backend not selected or invalid.  Assuming PyTorch for now.
/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)
        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [ ]:
# unit-level cr
GATE_hat_list = np.zeros((repeat_num, len(ramps)))
     
for seed in tqdm(range(repeat_num)):
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_nodes,))   
    
    for num_step in range(len(ramps)):
        print(f"Seed: {seed}")
        torch.manual_seed(2)
        p_list = [0, ramps[num_step]]
        g_feat_list = []
        y_list = []
        
        for p in p_list:
            z = (rollout_index < np.quantile(rollout_index, p))
            
            nx.set_node_attributes(g, 0, "z")            
            nx.set_node_attributes(g, {unit:1 for unit in range(num_nodes) if z[unit]}, "z")
            po_linear_model(g, gamma = 1)
            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
            
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/incre_2step_CR_unit.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.00, Loss: 8.1175
Epoch 0, Treat Prop: 0.02, Loss: 2.3776
Epoch 50, Treat Prop: 0.00, Loss: 0.1629
Epoch 50, Treat Prop: 0.02, Loss: 0.1791
Epoch 100, Treat Prop: 0.00, Loss: 0.0245
Epoch 100, Treat Prop: 0.02, Loss: 0.0383
Epoch 150, Treat Prop: 0.00, Loss: 0.0144
Epoch 150, Treat Prop: 0.02, Loss: 0.0220
Epoch 200, Treat Prop: 0.00, Loss: 0.0129
Epoch 200, Treat Prop: 0.02, Loss: 0.0179
Epoch 250, Treat Prop: 0.00, Loss: 0.0123
Epoch 250, Treat Prop: 0.02, Loss: 0.0157
Epoch 300, Treat Prop: 0.00, Loss: 0.0119
Epoch 300, Treat Prop: 0.02, Loss: 0.0146
Epoch 350, Treat Prop: 0.00, Loss: 0.0117
Epoch 350, Treat Prop: 0.02, Loss: 0.0140
0.01368899829685688
Seed: 0
Epoch 0, Treat Prop: 0.00, Loss: 8.1203
Epoch 0, Treat Prop: 0.05, Loss: 2.3876
Epoch 50, Treat Prop: 0.00, Loss: 0.1533
Epoch 50, Treat Prop: 0.05, Loss: 0.2205
Epoch 100, Treat Prop: 0.00, Loss: 0.0277
Epoch 100, Treat Prop: 0.05, Loss: 0.0637
Epoch 150, Treat Prop: 0.00, Loss: 0.0175
Epoch 150, Treat P

  0%|          | 1/1000 [00:23<6:30:33, 23.46s/it]

...

0.056428808718919754
Seed: 363
Epoch 0, Treat Prop: 0.00, Loss: 8.1070
Epoch 0, Treat Prop: 0.02, Loss: 2.3829
Epoch 50, Treat Prop: 0.00, Loss: 0.1626
Epoch 50, Treat Prop: 0.02, Loss: 0.1827
Epoch 100, Treat Prop: 0.00, Loss: 0.0234
Epoch 100, Treat Prop: 0.02, Loss: 0.0389
Epoch 150, Treat Prop: 0.00, Loss: 0.0141
Epoch 150, Treat Prop: 0.02, Loss: 0.0225
Epoch 200, Treat Prop: 0.00, Loss: 0.0128
Epoch 200, Treat Prop: 0.02, Loss: 0.0183
Epoch 250, Treat Prop: 0.00, Loss: 0.0123
Epoch 250, Treat Prop: 0.02, Loss: 0.0162
Epoch 300, Treat Prop: 0.00, Loss: 0.0120
Epoch 300, Treat Prop: 0.02, Loss: 0.0152
Epoch 350, Treat Prop: 0.00, Loss: 0.0118
Epoch 350, Treat Prop: 0.02, Loss: 0.0146
0.014206498861312866
Seed: 363
Epoch 0, Treat Prop: 0.00, Loss: 8.1155
Epoch 0, Treat Prop: 0.05, Loss: 2.3926
Epoch 50, Treat Prop: 0.00, Loss: 0.1546
Epoch 50, Treat Prop: 0.05, Loss: 0.2221
Epoch 100, Treat Prop: 0.00, Loss: 0.0276
Epoch 100, Treat Prop: 0.05, Loss: 0.0657
Epoch 150, Treat Prop: 0.0

 36%|███▋      | 364/1000 [2:46:44<6:24:53, 36.31s/it]

0.05461445450782776
Seed: 364
Epoch 0, Treat Prop: 0.00, Loss: 8.1159
Epoch 0, Treat Prop: 0.02, Loss: 2.3752
Epoch 50, Treat Prop: 0.00, Loss: 0.1579
Epoch 50, Treat Prop: 0.02, Loss: 0.1767
Epoch 100, Treat Prop: 0.00, Loss: 0.0246
Epoch 100, Treat Prop: 0.02, Loss: 0.0387
Epoch 150, Treat Prop: 0.00, Loss: 0.0149
Epoch 150, Treat Prop: 0.02, Loss: 0.0227
Epoch 200, Treat Prop: 0.00, Loss: 0.0134
Epoch 200, Treat Prop: 0.02, Loss: 0.0185
Epoch 250, Treat Prop: 0.00, Loss: 0.0126
Epoch 250, Treat Prop: 0.02, Loss: 0.0163
Epoch 300, Treat Prop: 0.00, Loss: 0.0122
Epoch 300, Treat Prop: 0.02, Loss: 0.0151
Epoch 350, Treat Prop: 0.00, Loss: 0.0119
Epoch 350, Treat Prop: 0.02, Loss: 0.0145
0.014249159023165703
Seed: 364
Epoch 0, Treat Prop: 0.00, Loss: 8.1204
Epoch 0, Treat Prop: 0.05, Loss: 2.4003
Epoch 50, Treat Prop: 0.00, Loss: 0.1534
Epoch 50, Treat Prop: 0.05, Loss: 0.2197
Epoch 100, Treat Prop: 0.00, Loss: 0.0279
Epoch 100, Treat Prop: 0.05, Loss: 0.0633
Epoch 150, Treat Prop: 0.00

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 50, Treat Prop: 0.00, Loss: 0.3542
Epoch 50, Treat Prop: 0.50, Loss: 1.0151
Epoch 100, Treat Prop: 0.00, Loss: 0.2167
Epoch 100, Treat Prop: 0.50, Loss: 0.5368
